In [3]:
import pandas as pd

# Giả sử 'data' là tập dữ liệu lịch sử với các cột là các đặc trưng và sản lượng điện tái tạo
params = pd.read_csv("/home/p06lab/solar_forecasting_kNN_KDE/Task 1/params.csv") # Tập dữ liệu lịch sử (cần được cung cấp)
train_size = int(len(params)*0.8)
test_size = len(params) - train_size
print(train_size, test_size)

22752 5688


In [7]:
params = params.drop(columns=['ZONEID', 'TIMESTAMP'])

In [5]:
label = pd.read_csv("/home/p06lab/solar_forecasting_kNN_KDE/Task 1/labels.csv")
label
Y = label.drop(columns=['ZONEID', 'TIMESTAMP'])

,POWER
0,0.754103
1,0.555000
2,0.438397
3,0.145449
4,0.111987
...,...
26275,0.000000
26276,0.008025
26277,0.240075
26278,0.600100


In [8]:
x_train = params[:train_size]
x_test  = params[train_size:]
y_train = Y[:train_size]
y_test  = Y[train_size:]

In [10]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.stats import gaussian_kde

# Chọn số lượng hàng xóm k
k = 200

# Tạo mô hình k-NN
knn = NearestNeighbors(n_neighbors=k)
knn.fit(x_train, y_train)

NearestNeighbors(n_neighbors=200)

In [25]:
x_test[:1][:].shape

(1, 12)

In [35]:

# Điểm dữ liệu mới cần dự báo
new_point = x_test[:1] # Điểm dữ liệu mới (cần được cung cấp)
# Tìm k hàng xóm gần nhất
distances, indices = knn.kneighbors(new_point)

In [36]:
print(distances, indices)

[[ 55897.79343704  63916.1575703  114271.01339977 119781.14760996
  128561.75550352 146897.98758442 149280.17186674 150575.38954775
  154864.12247782 160760.08226173 164648.01372263 177638.25384189
  178851.27590689 197918.06728624 215740.09941419 216153.64392526
  227013.91411775 241100.26290336 254765.50084278 257866.69327132
  263552.21815157 266820.22140471 275339.4818426  296860.71872258
  297860.29946706 308301.08887954 312550.26507958 314807.33422592
  315908.29599942 327280.24881154 337839.72467642 339264.73477845
  345142.96688025 347580.31549879 348395.98787109 353544.45515992
  357789.7842237  358436.40416957 374270.52949119 378853.35756665
  392821.47839966 397150.00304945 398059.80326982 400768.50984474
  400957.4209431  411301.89017376 414297.29869931 425077.83717749
  426226.2691182  429276.20929427 432615.61090778 432962.85863219
  436188.38210854 436461.61774984 436807.83834808 447511.57736459
  459116.53220664 459122.53490141 464848.65945018 472818.19988961
  473320.6

In [39]:
# Kiểm tra chỉ số hàng xóm để đảm bảo chúng nằm trong phạm vi của DataFrame x_test
indices = indices[0]
valid_indices = [i for i in indices if i < len(x_test)]

# Lấy các điểm dữ liệu hàng xóm gần nhất từ DataFrame x_test theo các chỉ số hợp lệ
neighbors_output = x_test.iloc[valid_indices]

print("Neighbors output:")
print(neighbors_output)

Neighbors output:
          VAR78     VAR79      VAR134     VAR157    VAR164    VAR165  \
23040  0.058922  0.001195  95144.0000  45.898041  0.398926 -3.396579   
23064  0.107483  0.001481  94607.0000  44.814056  0.438400 -2.795505   
23088  0.100204  0.001045  92997.0000  46.424408  0.412109 -2.205783   
23160  0.021240  0.000000  93066.0625  47.333908  0.251648 -1.703129   
24144  0.094498  0.000000  94803.5000  62.702789  1.000000 -4.931926   
...         ...       ...         ...        ...       ...       ...   
23592  0.101768  0.000047  92983.5625  60.513489  0.457308 -5.772902   
23352  0.026276  0.010511  94023.6875  39.044312  0.204575  1.499696   
25392  0.014221  0.000751  93192.8125  45.628891  0.149338 -0.973663   
23472  0.062961  0.000118  95008.5000  51.506073  0.452550 -5.228386   
25944  0.029654  0.000000  94280.5625  55.760635  0.335785  2.875569   

         VAR166      VAR167     VAR169       VAR175     VAR178  VAR228  
23040 -0.065736  296.590820  3289240.0  1235

 -Đảm bảo rằng bạn chỉ sử dụng một đặc trưng khi áp dụng KDE.
- Nếu bạn muốn áp dụng KDE cho tất cả các đặc trưng, bạn cần thực hiện KDE riêng biệt cho từng đặc trưng.

In [44]:
# Lấy các điểm dữ liệu hàng xóm gần nhất từ DataFrame x_test theo các chỉ số hợp lệ
neighbors_output = x_test.iloc[valid_indices].values.T  # Chuyển vị dữ liệu

# Áp dụng KDE để ước lượng mật độ xác suất
kde = gaussian_kde(neighbors_output)

# Tạo một phạm vi giá trị để tính mật độ xác suất
value_range = np.linspace(neighbors_output.min(), neighbors_output.max(), 1000)

# Tính mật độ xác suất
prob_density = kde(value_range)

# Vẽ đồ thị mật độ xác suất (tùy chọn)
plt.plot(value_range, prob_density)
plt.show()

ValueError: points have dimension 1, dataset has dimension 12

In [ ]:


# Rút ra 99 phân vị từ mật độ xác suất
quantiles = np.percentile(neighbors_output, np.arange(1, 100))

# Dự báo khoảng với mức độ tin cậy 95%
confidence_interval = [np.percentile(neighbors_output, 2.5), np.percentile(neighbors_output, 97.5)]

# In kết quả
print("Dự báo khoảng với mức độ tin cậy 95%:", confidence_interval)

In [ ]:

# # Điểm dữ liệu mới cần dự báo
# new_point = ... # Điểm dữ liệu mới (cần được cung cấp)

# # Tìm k hàng xóm gần nhất
# distances, indices = knn.kneighbors([new_point])

# Lấy sản lượng điện tái tạo từ k hàng xóm gần nhất
neighbors_output = x_test.iloc[indices[0], -1].values

# Áp dụng KDE để ước lượng mật độ xác suất
kde = gaussian_kde(neighbors_output)

# Tạo một phạm vi giá trị để tính mật độ xác suất
value_range = np.linspace(neighbors_output.min(), neighbors_output.max(), 1000)

# Tính mật độ xác suất
prob_density = kde(value_range)

# Vẽ đồ thị mật độ xác suất (tùy chọn)
import matplotlib.pyplot as plt
plt.plot(value_range, prob_density)
plt.show()

# Rút ra 99 phân vị từ mật độ xác suất
quantiles = np.percentile(neighbors_output, np.arange(1, 100))

# Dự báo khoảng với mức độ tin cậy 95%
confidence_interval = [np.percentile(neighbors_output, 2.5), np.percentile(neighbors_output, 97.5)]

# In kết quả
print("Dự báo khoảng với mức độ tin cậy 95%:", confidence_interval)